Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.10.1


In [18]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
PREPROCESSDATA_PATH = WORKSPACE_PATH+'/preprocessData'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [19]:
labels = [{'name':'Hello', 'id':1},
        {'name':'Thank you', 'id':2},
        {'name':'Yes', 'id':3},
        {'name':'Ok', 'id':4},
        {'name':'Nice', 'id':5}]

with open(PREPROCESSDATA_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [3]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {PREPROCESSDATA_PATH + '/label_map.pbtxt'} -o {PREPROCESSDATA_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {PREPROCESSDATA_PATH + '/label_map.pbtxt'} -o {PREPROCESSDATA_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/preprocessData/train.record
Successfully created the TFRecord file: Tensorflow/workspace/preprocessData/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [3]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [7]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [20]:
import os
import shutil

In [21]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_PATH = 'Tensorflow/workspace/pre-trained-models'
MODEL_PATH = 'Tensorflow/workspace/models'


In [22]:
os.makedirs(os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME), exist_ok=True)
shutil.copy(
    os.path.join(PRETRAINED_MODEL_PATH, 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'pipeline.config'),
    os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME)
)

'Tensorflow/workspace/models\\my_ssd_mobnet\\pipeline.config'

# 5. Update Config For Transfer Learning

In [23]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [24]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [25]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [26]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [27]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [28]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= PREPROCESSDATA_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [PREPROCESSDATA_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = PREPROCESSDATA_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [PREPROCESSDATA_PATH + '/test.record']

In [29]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [30]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=20000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=20000


# 7. Load Train Model From Checkpoint

In [31]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [32]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-19')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [33]:
import cv2 
import numpy as np

In [35]:
category_index = label_map_util.create_category_index_from_labelmap(PREPROCESSDATA_PATH+'/label_map.pbtxt')

In [36]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.3,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:


while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.3,
                agnostic_mode=False)

    ### ✨✨ Add background blur from here ✨✨
    if num_detections > 0 and detections['detection_scores'][0] > 0.3:
        # Get box coordinates
        box = detections['detection_boxes'][0]  # first detected hand
        ymin, xmin, ymax, xmax = box

        # Convert normalized coordinates to pixel values
        (left, right, top, bottom) = (xmin * width, xmax * width, ymin * height, ymax * height)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)

        # Apply Gaussian blur to entire frame
        blurred_frame = cv2.GaussianBlur(image_np_with_detections, (55, 55), 0)

        # Paste the original (sharp) detected hand region
        blurred_frame[top:bottom, left:right] = image_np_with_detections[top:bottom, left:right]

        # Show the final blurred frame
        cv2.imshow('object detection', cv2.resize(blurred_frame, (800, 600)))
    else:
        # If no detection or detection score is too low, show normal frame
        cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    ### ✨✨ Blur ends here ✨✨
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

KeyboardInterrupt: 

In [ ]:


while True: 
    ret, frame = cap.read()
    if not ret:
        break

    image_np = np.array(frame)

    # Make a copy of the original frame
    original_frame = image_np.copy()

    # Run detection
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1

    # Step 1: Blur the original frame
    blurred_frame = cv2.GaussianBlur(original_frame, (55, 55), 0)

    # Step 2: If any detections with good confidence
    if num_detections > 0 and detections['detection_scores'][0] > 0.3:
        # Get first detection bounding box
        box = detections['detection_boxes'][0]
        ymin, xmin, ymax, xmax = box

        # Convert normalized coordinates to pixel coordinates
        left, right, top, bottom = (xmin * width, xmax * width, ymin * height, ymax * height)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)

        # Keep hand region sharp
        blurred_frame[top:bottom, left:right] = original_frame[top:bottom, left:right]

    # Step 3: Draw the bounding box and label on the final blurred frame
    viz_utils.visualize_boxes_and_labels_on_image_array(
                blurred_frame,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.3,
                agnostic_mode=False)

    # Step 4: Display the result
    cv2.imshow('Sign Language Detection', cv2.resize(blurred_frame, (800, 600)))

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 